Created on Fri March 02 14:42:35 2020

@author:Patricia de Melo Maia
cohort: 5 - Valencia
    
    A) Introduction:
    Apprentice Chef, Inc. is an innovative company with a unique spin on cooking at home. 
    After three years serving customers across the San Francisco Bay Area, the executives 
    at Apprentice Chef have come to realize that over 90% of their revenue comes from customers
    that have been ordering meal sets for 12 months or less. Given this information, they would
    like to better understand how much revenue to expect from each customer within their first 
    year of orders. Thus, they have hired you on a full-timecontract to analyze their data, 
    develop your top insights, and build a machine learning model to predict revenue over the 
    first year of each customer’s life cycle.

In [1]:
##########################################################################################################
#Inicialization #########################################################################################
##########################################################################################################
# importing libraries
import pandas as pd # data science essentials
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # enhanced data visualization
import statsmodels.formula.api as smf # regression modeling
from sklearn.model_selection import train_test_split # train/test split
import sklearn.linear_model # linear models
import gender_guesser.detector as gender # gender detection
import sklearn.neighbors # KNN for Regression
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
          
# setting pandas print options
#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)

# specifying file name
file = 'Apprentice_Chef_Dataset.xlsx'

# reading the file 
df = pd.read_excel(file)


In [2]:
##########################################################################################################
#Dataset Exploration ####################################################################################
##########################################################################################################
#First lines
#df.head(n = 5)

# Information about each variable
#df.info()

# descriptive statistics
#df.describe().round(2)

#Checking Missing Value Analysis and Imputation
#df.isnull().sum()

First I imported the libraries, after, uploaded the file and then, and, then, check the information in the dataset. Checking the first 5 lines of dataset we see there are more information in the column name, like the relation between some users. Checking the info of the dataset we can see that most of the information are interger(22), there are three float and four objects. Checking the descrictive statistic information, we can see that all the user mande contact with customer service. Also, most people is registered with a mobile phone.

In [3]:
##########################################################################################################
#Linear Regression #######################################################################################
##########################################################################################################
# building a base model will all varibles

lm_revenue = smf.ols(formula = """REVENUE ~
                                df['CROSS_SELL_SUCCESS'] +
                                df['TOTAL_MEALS_ORDERED'] +
                                df['UNIQUE_MEALS_PURCH'] +
                                df['CONTACTS_W_CUSTOMER_SERVICE'] +
                                df['PRODUCT_CATEGORIES_VIEWED'] +
                                df['AVG_TIME_PER_SITE_VISIT'] +
                                df['MOBILE_NUMBER'] +
                                df['CANCELLATIONS_BEFORE_NOON'] +
                                df['CANCELLATIONS_AFTER_NOON'] +
                                df['TASTES_AND_PREFERENCES'] +
                                df['MOBILE_LOGINS'] +
                                df['PC_LOGINS'] +
                                df['WEEKLY_PLAN'] +
                                df['EARLY_DELIVERIES'] +
                                df['LATE_DELIVERIES'] +
                                df['PACKAGE_LOCKER'] +
                                df['REFRIGERATED_LOCKER'] +
                                df['FOLLOWED_RECOMMENDATIONS_PCT'] +
                                df['AVG_PREP_VID_TIME'] +
                                df['LARGEST_ORDER_SIZE'] +
                                df['MASTER_CLASSES_ATTENDED'] +
                                df['MEDIAN_MEAL_RATING'] +
                                df['AVG_CLICKS_PER_VISIT'] +
                                df['TOTAL_PHOTOS_VIEWED']""",
                                data = df)
results = lm_revenue.fit()
# printing the results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                REVENUE   R-squared:                       0.641
Model:                            OLS   Adj. R-squared:                  0.636
Method:                 Least Squares   F-statistic:                     142.7
Date:                Sat, 07 Mar 2020   Prob (F-statistic):               0.00
Time:                        23:45:16   Log-Likelihood:                -15460.
No. Observations:                1946   AIC:                         3.097e+04
Df Residuals:                    1921   BIC:                         3.111e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Inte

I runned a linear regression with all the intergir (22) and float(3) to check how are the p-values between them.
I found p-values to some of theses variables. Instead of drop them, I will flag them as binary (1-use the feature , 0-did not use it) to check if the information existence is relevante to my model. Unfortunately, this approch did not improve my model. Now I will create new variables. 

In [4]:
##########################################################################################################
#Creating new variable #######################################################################################
##########################################################################################################

##### Dividing the avg time per site visit by total orders.
df["AVG_TIME_PER_SITE_VISIT_PER_ORDER"]= df['AVG_TIME_PER_SITE_VISIT']/df['TOTAL_MEALS_ORDERED']

##### Dividing the avg prep vid time by total orders.
df['AVG_PREP_VID_TIME_PER_ORDER'] = df['AVG_PREP_VID_TIME']/df['TOTAL_MEALS_ORDERED']

##### Dividing the avg clicks per visit by total orders.
df['AVG_CLICKS_PER_VISIT_PER_ORDER'] = df['AVG_CLICKS_PER_VISIT']/df['TOTAL_MEALS_ORDERED']

##### Dividing PRODUCT CATEGORIES VIEWED by total orders.
df['PRODUCT_CATEGORIES_VIEWED_PER_ORDER'] = df['PRODUCT_CATEGORIES_VIEWED']/df['TOTAL_MEALS_ORDERED']

##### NUMBER OF RECOMMENDATION
df["NUMBER_OF_RECOMMENDATION"]= df['FOLLOWED_RECOMMENDATIONS_PCT']/100 * df['TOTAL_MEALS_ORDERED']



In [5]:
##########################################################################################################
#Ploting Histogram #######################################################################################
##########################################################################################################

objectcol = ['NAME', 'EMAIL', 'FIRST_NAME', 'FAMILY_NAME'] #object variables

#### commented for save time and output
#for col in df.columns:
#    if col not in objectcol:
#        fig, ax = plt.subplots(figsize = (6, 4))
#        sns.distplot(df[col])                 
#        plt.xlabel(col)


In [6]:
##########################################################################################################
#Feature Engineering ####################################################################################
##########################################################################################################

#thresholds - Base on Histogram analysis
#REVENUE
#CROSS_SELL_SUCCESS
TOTAL_MEALS_ORDERED_HI = 200
UNIQUE_MEALS_PURCH_HI = 9
CONTACTS_W_CUSTOMER_SERVICE_HI= 12
CONTACTS_W_CUSTOMER_SERVICE_LO= 3
PRODUCT_CATEGORIES_VIEWED_HI = 10
PRODUCT_CATEGORIES_VIEWED_LO = 2
AVG_TIME_PER_SITE_VISIT_HI= 240
#MOBILE_NUMBER
CANCELLATIONS_BEFORE_NOON_HI = 5
CANCELLATIONS_AFTER_NOON_HI = 2
#TASTES_AND_PREFERENCES 
PC_LOGINS_HI= 7
PC_LOGINS_LO= 4
MOBILE_LOGINS_HI= 2
WEEKLY_PLAN_HI= 15
EARLY_DELIVERIES_HI = 4
LATE_DELIVERIES_HI= 7
#PACKAGE_LOCKER => is already binary
#REFRIGERATED_LOCKER  => is already binary
#FOLLOWED_RECOMMENDATIONS_PCT
AVG_PREP_VID_TIME_HI= 280
LARGEST_ORDER_SIZE_HI = 7.5
MASTER_CLASSES_ATTENDED_HI = 2
#MEDIAN_MEAL_RATING 
AVG_CLICKS_PER_VISIT_HI=18
AVG_CLICKS_PER_VISIT_LO=8
TOTAL_PHOTOS_VIEWED_HI= 200
AVG_TIME_PER_SITE_VISIT_PER_ORDER_HI= 8
AVG_PREP_VID_TIME_PER_ORDER_HI = 7.5
AVG_CLICKS_PER_VISIT_PER_ORDER_HI = 0.7
PRODUCT_CATEGORIES_VIEWED_PER_ORDER_HI= 0.25
NUMBER_OF_RECOMMENDATION_HI= 80


#TOTAL_MEALS_ORDERED ############################################################################################

df['T_TOTAL_MEALS_ORDERED'] = 0
condition_hi = df.loc[0:,'T_TOTAL_MEALS_ORDERED'][df['TOTAL_MEALS_ORDERED'] > TOTAL_MEALS_ORDERED_HI]

df['T_TOTAL_MEALS_ORDERED'].replace(to_replace = condition_hi,value = 1,inplace = True)


#UNIQUE_MEALS_PURCH ##############################################################################################

df['T_UNIQUE_MEALS_PURCH'] = 0
condition_hi = df.loc[0:,'T_UNIQUE_MEALS_PURCH'][df['UNIQUE_MEALS_PURCH'] > UNIQUE_MEALS_PURCH_HI]

df['T_UNIQUE_MEALS_PURCH'].replace(to_replace = condition_hi,value = 1,inplace = True)

#CONTACTS_W_CUSTOMER_SERVICE #####################################################################################

df['T_CONTACTS_W_CUSTOMER_SERVICE'] = 0
condition_hi = df.loc[0:,'T_CONTACTS_W_CUSTOMER_SERVICE'][df['CONTACTS_W_CUSTOMER_SERVICE'] > CONTACTS_W_CUSTOMER_SERVICE_HI]
condition_lo = df.loc[0:,'T_CONTACTS_W_CUSTOMER_SERVICE'][df['CONTACTS_W_CUSTOMER_SERVICE'] < CONTACTS_W_CUSTOMER_SERVICE_LO]


df['T_CONTACTS_W_CUSTOMER_SERVICE'].replace(to_replace = condition_hi,value = 1,inplace = True)
df['T_CONTACTS_W_CUSTOMER_SERVICE'].replace(to_replace = condition_lo,value = 1,inplace = True)

#PRODUCT_CATEGORIES_VIEWED - HIGHER AND LOWER LIMITS ##############################################################

df['T_PRODUCT_CATEGORIES_VIEWED'] = 0
condition_hi = df.loc[0:,'T_PRODUCT_CATEGORIES_VIEWED'][df['PRODUCT_CATEGORIES_VIEWED'] > PRODUCT_CATEGORIES_VIEWED_HI]
condition_lo = df.loc[0:,'T_PRODUCT_CATEGORIES_VIEWED'][df['PRODUCT_CATEGORIES_VIEWED'] < PRODUCT_CATEGORIES_VIEWED_LO]

df['T_PRODUCT_CATEGORIES_VIEWED'].replace(to_replace = condition_hi,value = 1,inplace = True)
df['T_PRODUCT_CATEGORIES_VIEWED'].replace(to_replace = condition_lo,value = 1,inplace = True)

# AVG_TIME_PER_SITE_VISIT #########################################################################################

df['T_AVG_TIME_PER_SITE_VISIT'] = 0
condition_hi = df.loc[0:,'T_AVG_TIME_PER_SITE_VISIT'][df['AVG_TIME_PER_SITE_VISIT'] > AVG_TIME_PER_SITE_VISIT_HI]

df['T_AVG_TIME_PER_SITE_VISIT'].replace(to_replace = condition_hi,value = 1,inplace = True)


#CANCELLATIONS_BEFORE_NOON #########################################################################################

df['T_CANCELLATIONS_BEFORE_NOON'] = 0
condition_hi = df.loc[0:,'T_CANCELLATIONS_BEFORE_NOON'][df['CANCELLATIONS_BEFORE_NOON'] > CANCELLATIONS_BEFORE_NOON_HI]

df['T_CANCELLATIONS_BEFORE_NOON'].replace(to_replace = condition_hi,value = 1,inplace = True)


#CANCELLATIONS_AFTER_NOON ###########################################################################################

df['T_CANCELLATIONS_AFTER_NOON'] = 0
condition_hi = df.loc[0:,'T_CANCELLATIONS_AFTER_NOON'][df['CANCELLATIONS_AFTER_NOON'] > CANCELLATIONS_AFTER_NOON_HI]

df['T_CANCELLATIONS_AFTER_NOON'].replace(to_replace = condition_hi,value = 1,inplace = True)

#PC_LOGINS - HIGHER AND LOWER LIMITS #############################################################################

df['T_PC_LOGINS'] = 0
condition_hi = df.loc[0:,'T_PC_LOGINS'][df['PC_LOGINS'] > PC_LOGINS_HI]
condition_lo = df.loc[0:,'T_PC_LOGINS'][df['PC_LOGINS'] < PC_LOGINS_LO]

df['T_PC_LOGINS'].replace(to_replace = condition_hi,value = 1,inplace = True)
df['T_PC_LOGINS'].replace(to_replace = condition_lo,value = 1,inplace = True)

#MOBILE_LOGINS #####################################################################################################
df['T_MOBILE_LOGINS'] = 0
condition = df.loc[0:,'T_MOBILE_LOGINS'][df['MOBILE_LOGINS'] > MOBILE_LOGINS_HI]

df['T_MOBILE_LOGINS'].replace(to_replace = condition_hi,value = 1,inplace = True)


# WEEKLY_PLAN ######################################################################################################
df['T_WEEKLY_PLAN'] = 0
condition = df.loc[0:,'T_WEEKLY_PLAN'][df['WEEKLY_PLAN'] > WEEKLY_PLAN_HI]

df['T_WEEKLY_PLAN'].replace(to_replace = condition_hi,value = 1,inplace = True)


#EARLY_DELIVERIES ##################################################################################################
df['T_EARLY_DELIVERIES'] = 0
condition = df.loc[0:,'T_EARLY_DELIVERIES'][df['EARLY_DELIVERIES'] > EARLY_DELIVERIES_HI]

df['T_EARLY_DELIVERIES'].replace(to_replace = condition_hi,value = 1,inplace = True)


#LATE_DELIVERIES ##################################################################################################
df['T_LATE_DELIVERIES'] = 0
condition_hi = df.loc[0:,'T_LATE_DELIVERIES'][df['LATE_DELIVERIES'] > LATE_DELIVERIES_HI]

df['T_LATE_DELIVERIES'].replace(to_replace = condition_hi,value = 1,inplace = True)


#AVG_PREP_VID_TIME ###########################################################################################
df['T_AVG_PREP_VID_TIME'] = 0
condition_hi = df.loc[0:,'T_AVG_PREP_VID_TIME'][df['AVG_PREP_VID_TIME'] > AVG_PREP_VID_TIME_HI]

df['T_AVG_PREP_VID_TIME'].replace(to_replace = condition_hi,value = 1,inplace = True)

#LARGEST_ORDER_SIZE ###########################################################################################
df['T_LARGEST_ORDER_SIZE'] = 0
condition_hi = df.loc[0:,'T_LARGEST_ORDER_SIZE'][df['LARGEST_ORDER_SIZE'] > LARGEST_ORDER_SIZE_HI]

df['T_LARGEST_ORDER_SIZE'].replace(to_replace = condition_hi,value = 1,inplace = True)


#MASTER_CLASSES_ATTENDED ###########################################################################################
df['T_MASTER_CLASSES_ATTENDED'] = 0
condition_HI = df.loc[0:,'T_MASTER_CLASSES_ATTENDED'][df['MASTER_CLASSES_ATTENDED'] > MASTER_CLASSES_ATTENDED_HI]

df['T_MASTER_CLASSES_ATTENDED'].replace(to_replace = condition_hi,value = 1,inplace = True)


# AVG_CLICKS_PER_VISIT - HIGHER AND LOWER LIMITS ###################################################################
df['T_AVG_CLICKS_PER_VISIT'] = 0
condition_hi = df.loc[0:,'T_AVG_CLICKS_PER_VISIT'][df['AVG_CLICKS_PER_VISIT'] > AVG_CLICKS_PER_VISIT_HI]
condition_lo = df.loc[0:,'T_AVG_CLICKS_PER_VISIT'][df['AVG_CLICKS_PER_VISIT'] < AVG_CLICKS_PER_VISIT_LO]

df['T_AVG_CLICKS_PER_VISIT'].replace(to_replace = condition_hi,value = 1,inplace = True)
df['T_AVG_CLICKS_PER_VISIT'].replace(to_replace = condition_lo,value = 1,inplace = True)


# TOTAL_PHOTOS_VIEWED ##############################################################################################

df['T_TOTAL_PHOTOS_VIEWED'] = 0
condition_hi = df.loc[0:,'T_TOTAL_PHOTOS_VIEWED'][df['TOTAL_PHOTOS_VIEWED'] > TOTAL_PHOTOS_VIEWED_HI]

df['T_TOTAL_PHOTOS_VIEWED'].replace(to_replace = condition_hi,
                                           value      = 1,
                                           inplace    = True)

#AVG_TIME_PER_SITE_VISIT_PER_ORDER #################################################################################


df['T_TOTAL_PHOTOS_VIEWED'] = 0
condition_hi = df.loc[0:,'T_TOTAL_PHOTOS_VIEWED'][df['TOTAL_PHOTOS_VIEWED'] > TOTAL_PHOTOS_VIEWED_HI]

df['T_TOTAL_PHOTOS_VIEWED'].replace(to_replace = condition_hi,
                                           value      = 1,
                                           inplace    = True)

#AVG_PREP_VID_TIME_PER_ORDER #######################################################################################

df['T_AVG_PREP_VID_TIME_PER_ORDER'] = 0
condition_hi = df.loc[0:,'T_AVG_PREP_VID_TIME_PER_ORDER'][df['AVG_PREP_VID_TIME_PER_ORDER'] > AVG_PREP_VID_TIME_PER_ORDER_HI]

df['T_AVG_PREP_VID_TIME_PER_ORDER'].replace(to_replace = condition_hi,
                                           value      = 1,
                                           inplace    = True)
#AVG_CLICKS_PER_VISIT_PER_ORDER ####################################################################################

df['T_AVG_CLICKS_PER_VISIT_PER_ORDER'] = 0
condition_hi = df.loc[0:,'T_AVG_CLICKS_PER_VISIT_PER_ORDER'][df['AVG_CLICKS_PER_VISIT_PER_ORDER'] > AVG_CLICKS_PER_VISIT_PER_ORDER_HI]

df['T_AVG_CLICKS_PER_VISIT_PER_ORDER'].replace(to_replace = condition_hi,
                                           value      = 1,
                                           inplace    = True)
#PRODUCT_CATEGORIES_VIEWED_PER_ORDER ###############################################################################

df['T_PRODUCT_CATEGORIES_VIEWED_PER_ORDER'] = 0
condition_hi = df.loc[0:,'T_PRODUCT_CATEGORIES_VIEWED_PER_ORDER'][df['PRODUCT_CATEGORIES_VIEWED_PER_ORDER'] > PRODUCT_CATEGORIES_VIEWED_PER_ORDER_HI]

df['T_PRODUCT_CATEGORIES_VIEWED_PER_ORDER'].replace(to_replace = condition_hi,
                                           value      = 1,
                                           inplace    = True)
#NUMBER_OF_RECOMMENDATION ##########################################################################################

df['T_NUMBER_OF_RECOMMENDATION'] = 0
condition_hi = df.loc[0:,'T_NUMBER_OF_RECOMMENDATION'][df['NUMBER_OF_RECOMMENDATION'] > NUMBER_OF_RECOMMENDATION_HI]

df['T_NUMBER_OF_RECOMMENDATION'].replace(to_replace = condition_hi,
                                           value      = 1,
                                           inplace    = True)

In [7]:
##########################################################################################################
#Ploting Scatterplots ####################################################################################
##########################################################################################################

colm = [ 'NAME','EMAIL', 'FIRST_NAME', 'FAMILY_NAME','MOBILE_NUMBER']
#
#### commented for save time and output
#for col in df.columns:
#    if col not in colm:
#        fig, ax = plt.subplots(figsize = (8, 6))
#        sns.scatterplot(x = df[col],
#                       y = df['REVENUE'],
#                        color = 'g')
#        plt.xlabel(col)
#       plt.savefig('Apprentice Scatterplots1 .png')


In [8]:
##########################################################################################################
#Feature Engineering ####################################################################################
##########################################################################################################
#thresholds - Base on scatterplot

#REVENUE
#CROSS_SELL_SUCCESS
TOTAL_MEALS_ORDERED_CHANGE = 120
UNIQUE_MEALS_PURCH_CHANGE = 9 
CONTACTS_W_CUSTOMER_SERVICE_CHANGE= 10
#PRODUCT_CATEGORIES_VIEWED
AVG_TIME_PER_SITE_VISIT_CHANGE= 190
#MOBILE_NUMBER
CANCELLATIONS_BEFORE_NOON_CHANGE = 8
CANCELLATIONS_AFTER_NOON_CHANGE = 2
#TASTES_AND_PREFERENCES 
#PC_LOGINS
#MOBILE_LOGINS
WEEKLY_PLAN_CHANGE= 15
#EARLY_DELIVERIES_HI = 4
LATE_DELIVERIES_CHANGE= 9
#PACKAGE_LOCKER => is already binary
#REFRIGERATED_LOCKER  => is already binary
#FOLLOWED_RECOMMENDATIONS_PCT
AVG_PREP_VID_TIME_CHANGE= 210
LARGEST_ORDER_SIZE_HI = 6
MASTER_CLASSES_ATTENDED_HI = 2
#MEDIAN_MEAL_RATING 
AVG_CLICKS_PER_VISIT_CHANGE= 10
TOTAL_PHOTOS_VIEWED_CHANGE= 300
AVG_TIME_PER_SITE_VISIT_PER_ORDER_CHARGE= 7.5
AVG_PREP_VID_TIME_PER_ORDER_CHARGE = 4
AVG_CLICKS_PER_VISIT_PER_ORDER_CHARGE= 0.6
PRODUCT_CATEGORIES_VIEWED_PER_ORDER_CHARGE= 0.3
NUMBER_OF_RECOMMENDATION_CHARGE= 60


#TOTAL_MEALS_ORDERED ###########################################################################################
df['TOTAL_MEALS_ORDERED_C'] = 0
condition = df.loc[0:,'TOTAL_MEALS_ORDERED_C']\
                  [df['TOTAL_MEALS_ORDERED'] > TOTAL_MEALS_ORDERED_CHANGE]

df['TOTAL_MEALS_ORDERED_C'].replace(to_replace = condition,value = 1, inplace= True)

#UNIQUE_MEALS_PURCH ###########################################################################################
df['UNIQUE_MEALS_PURCH_C'] = 0
condition = df.loc[0:,'UNIQUE_MEALS_PURCH_C']\
                  [df['UNIQUE_MEALS_PURCH'] > UNIQUE_MEALS_PURCH_CHANGE]

df['UNIQUE_MEALS_PURCH_C'].replace(to_replace = condition,value = 1, inplace= True)


#AVG_TIME_PER_SITE_VISIT ###########################################################################################
df['AVG_TIME_PER_SITE_VISIT_C'] = 0
condition = df.loc[0:,'AVG_TIME_PER_SITE_VISIT_C']\
                  [df['AVG_TIME_PER_SITE_VISIT'] > AVG_TIME_PER_SITE_VISIT_CHANGE]

df['AVG_TIME_PER_SITE_VISIT_C'].replace(to_replace = condition,value = 1, inplace= True)

##########################################################################################################
#WEEKLY_PLAN
#df['WEEKLY_PLAN_C'] = 0
#condition = df.loc[0:,'WEEKLY_PLAN_C']\
#                  [df['WEEKLY_PLAN'] > WEEKLY_PLAN_CHANGE]

#df['WEEKLY_PLAN_C'].replace(to_replace = condition,value = 1, inplace= True)
##########################################################################################################
#AVG_CLICKS_PER_VISIT
df['AVG_CLICKS_PER_VISIT_C'] = 0
condition = df.loc[0:,'AVG_CLICKS_PER_VISIT_C']\
                  [df['AVG_CLICKS_PER_VISIT'] > AVG_CLICKS_PER_VISIT_CHANGE]

df['AVG_CLICKS_PER_VISIT_C'].replace(to_replace = condition,value = 1, inplace= True)

##########################################################################################################
#AVG_PREP_VID_TIME
df['AVG_PREP_VID_TIME_C'] = 0
condition = df.loc[0:,'AVG_PREP_VID_TIME_C']\
                  [df['AVG_PREP_VID_TIME'] > AVG_PREP_VID_TIME_CHANGE]

df['AVG_PREP_VID_TIME_C'].replace(to_replace = condition,value = 1, inplace= True)

##########################################################################################################
#TOTAL_PHOTOS_VIEWED
df['TOTAL_PHOTOS_VIEWED_C'] = 0
condition = df.loc[0:,'TOTAL_PHOTOS_VIEWED_C']\
                  [df['TOTAL_PHOTOS_VIEWED'] >TOTAL_PHOTOS_VIEWED_CHANGE]

df['TOTAL_PHOTOS_VIEWED_C'].replace(to_replace = condition,value = 1, inplace= True)



In [9]:
##########################################################################################################
#Correlation #############################################################################################
##########################################################################################################

# Checking the correlation matrix
df_corr = df.corr().round(2)

# printing (Pearson) correlations with REVENUE
print(df_corr.loc[ : ,'REVENUE'].sort_values(ascending = False))

REVENUE                                  1.00
AVG_PREP_VID_TIME                        0.64
MEDIAN_MEAL_RATING                       0.61
TOTAL_MEALS_ORDERED                      0.60
TOTAL_PHOTOS_VIEWED                      0.47
AVG_PREP_VID_TIME_C                      0.47
MASTER_CLASSES_ATTENDED                  0.45
LARGEST_ORDER_SIZE                       0.44
TOTAL_MEALS_ORDERED_C                    0.42
T_TOTAL_PHOTOS_VIEWED                    0.41
TOTAL_PHOTOS_VIEWED_C                    0.39
NUMBER_OF_RECOMMENDATION                 0.37
T_TOTAL_MEALS_ORDERED                    0.35
T_NUMBER_OF_RECOMMENDATION               0.25
T_LARGEST_ORDER_SIZE                     0.21
T_MASTER_CLASSES_ATTENDED                0.21
T_AVG_PREP_VID_TIME                      0.20
UNIQUE_MEALS_PURCH_C                     0.17
T_UNIQUE_MEALS_PURCH                     0.17
AVG_TIME_PER_SITE_VISIT                  0.14
CONTACTS_W_CUSTOMER_SERVICE              0.10
T_AVG_TIME_PER_SITE_VISIT         

In [10]:
# Copying the file
df_explanatory = df.copy()

# formatting each explanatory variable for statsmodels
for val in df_explanatory:
    print(f"df['{val}']+") 

df['REVENUE']+
df['CROSS_SELL_SUCCESS']+
df['NAME']+
df['EMAIL']+
df['FIRST_NAME']+
df['FAMILY_NAME']+
df['TOTAL_MEALS_ORDERED']+
df['UNIQUE_MEALS_PURCH']+
df['CONTACTS_W_CUSTOMER_SERVICE']+
df['PRODUCT_CATEGORIES_VIEWED']+
df['AVG_TIME_PER_SITE_VISIT']+
df['MOBILE_NUMBER']+
df['CANCELLATIONS_BEFORE_NOON']+
df['CANCELLATIONS_AFTER_NOON']+
df['TASTES_AND_PREFERENCES']+
df['PC_LOGINS']+
df['MOBILE_LOGINS']+
df['WEEKLY_PLAN']+
df['EARLY_DELIVERIES']+
df['LATE_DELIVERIES']+
df['PACKAGE_LOCKER']+
df['REFRIGERATED_LOCKER']+
df['FOLLOWED_RECOMMENDATIONS_PCT']+
df['AVG_PREP_VID_TIME']+
df['LARGEST_ORDER_SIZE']+
df['MASTER_CLASSES_ATTENDED']+
df['MEDIAN_MEAL_RATING']+
df['AVG_CLICKS_PER_VISIT']+
df['TOTAL_PHOTOS_VIEWED']+
df['AVG_TIME_PER_SITE_VISIT_PER_ORDER']+
df['AVG_PREP_VID_TIME_PER_ORDER']+
df['AVG_CLICKS_PER_VISIT_PER_ORDER']+
df['PRODUCT_CATEGORIES_VIEWED_PER_ORDER']+
df['NUMBER_OF_RECOMMENDATION']+
df['T_TOTAL_MEALS_ORDERED']+
df['T_UNIQUE_MEALS_PURCH']+
df['T_CONTACTS_W_CUSTOMER_SERVI

In [11]:
lm_full = smf.ols(formula = """ REVENUE  ~  df['CROSS_SELL_SUCCESS']+
                                            
                                            
                                            
                                            
                                            df['TOTAL_MEALS_ORDERED']+
                                            df['UNIQUE_MEALS_PURCH']+
                                            df['CONTACTS_W_CUSTOMER_SERVICE']+
                                            df['PRODUCT_CATEGORIES_VIEWED']+
                                            
                                            
                                            
                                            df['CANCELLATIONS_AFTER_NOON']+
                                            
                                            
                                            df['MOBILE_LOGINS']+
                                            
                                            
                                            df['LATE_DELIVERIES']+
                                            
                                            
                                            df['FOLLOWED_RECOMMENDATIONS_PCT']+
                                            df['AVG_PREP_VID_TIME']+
                                            df['LARGEST_ORDER_SIZE']+
                                            df['MASTER_CLASSES_ATTENDED']+
                                            df['MEDIAN_MEAL_RATING']+
                                            df['AVG_CLICKS_PER_VISIT']+
                                            
                                            
                                            df['AVG_PREP_VID_TIME_PER_ORDER']+
                                            df['AVG_CLICKS_PER_VISIT_PER_ORDER']+
                                            
                                            df['NUMBER_OF_RECOMMENDATION']+
                                            
                                            df['T_UNIQUE_MEALS_PURCH']+
                                            
                                            df['T_PRODUCT_CATEGORIES_VIEWED']+
                                            df['T_AVG_TIME_PER_SITE_VISIT']+
                                            
                                            
                                            df['T_PC_LOGINS']+
                                            df['T_MOBILE_LOGINS']+
                                            
                                            df['T_EARLY_DELIVERIES']+
                                            df['T_LATE_DELIVERIES']+
                                            df['T_AVG_PREP_VID_TIME']+
                                            
                                            
                                            df['T_AVG_CLICKS_PER_VISIT']+
                                            df['T_TOTAL_PHOTOS_VIEWED']+
                                            df['T_AVG_PREP_VID_TIME_PER_ORDER']+
                                            df['T_AVG_CLICKS_PER_VISIT_PER_ORDER']+
                                            
                                           
                                            
                                            df['UNIQUE_MEALS_PURCH_C']+
                                            
                                            df['AVG_CLICKS_PER_VISIT_C']+
                                            df['AVG_PREP_VID_TIME_C']+
                                            df['TOTAL_PHOTOS_VIEWED_C']
                                                                    """,
                                            data = df_explanatory) 
results_full = lm_full.fit()

# printing the results
results_full.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                REVENUE   R-squared:                       0.685
Model:                            OLS   Adj. R-squared:                  0.680
Method:                 Least Squares   F-statistic:                     143.8
Date:                Sat, 07 Mar 2020   Prob (F-statistic):               0.00
Time:                        23:45:32   Log-Likelihood:                -15331.
No. Observations:                1946   AIC:                         3.072e+04
Df Residuals:                    1916   BIC:                         3.089e+04
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
==========================================================================================================
                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
Intercept                                424.2416    294.050      1.443      0.149    -152.450    1000.933
df['CROSS_SELL_SUCCESS']                 -69.2433     35.601     -1.945      0.052    -139.063       0.577
df['TOTAL_MEALS_ORDERED']                  2.0717      0.617      3.355      0.001       0.861       3.283
df['UNIQUE_MEALS_PURCH']                 -66.2995      6.122    -10.830      0.000     -78.306     -54.293
df['CONTACTS_W_CUSTOMER_SERVICE']        116.5224     10.358     11.250      0.000      96.209     136.836
df['PRODUCT_CATEGORIES_VIEWED']            8.7671      5.376      1.631      0.103      -1.777      19.311
df['CANCELLATIONS_AFTER_NOON']           -61.9703     33.984     -1.824      0.068    -128.620       4.679
df['MOBILE_LOGINS']                      -29.1135     27.906     -1.043      0.297     -83.843      25.615
df['LATE_DELIVERIES']                     12.5390      7.470      1.678      0.093      -2.112      27.190
df['FOLLOWED_RECOMMENDATIONS_PCT']        -1.0463      0.991     -1.055      0.291      -2.991       0.898
df['AVG_PREP_VID_TIME']                   10.8752      0.850     12.789      0.000       9.207      12.543
df['LARGEST_ORDER_SIZE']                 -81.3440     16.453     -4.944      0.000    -113.612     -49.076
df['MASTER_CLASSES_ATTENDED']            137.5602     26.978      5.099      0.000      84.650     190.470
df['MEDIAN_MEAL_RATING']                 208.7980     41.155      5.073      0.000     128.085     289.511
df['AVG_CLICKS_PER_VISIT']               -18.9022     13.988     -1.351      0.177     -46.336       8.532
df['AVG_PREP_VID_TIME_PER_ORDER']       -155.9522     25.124     -6.207      0.000    -205.225    -106.679
df['AVG_CLICKS_PER_VISIT_PER_ORDER']    1047.4912    232.756      4.500      0.000     591.009    1503.974
df['NUMBER_OF_RECOMMENDATION']             1.6089      1.045      1.540      0.124      -0.440       3.657
df['T_UNIQUE_MEALS_PURCH']               340.5680     84.854      4.014      0.000     174.152     506.984
df['T_PRODUCT_CATEGORIES_VIEWED']        145.9807     59.029      2.473      0.013      30.214     261.748
df['T_AVG_TIME_PER_SITE_VISIT']         -237.8010    139.084     -1.710      0.087    -510.572      34.970
df['T_PC_LOGINS']                       9.996e-13   1.63e-13      6.144      0.000     6.8e-13    1.32e-12
df['T_MOBILE_LOGINS']                  -4.728e-13   6.41e-14     -7.378      0.000   -5.98e-13   -3.47e-13
df['T_EARLY_DELIVERIES']                2.056e-13   5.21e-14      3.949      0.000    1.03e-13    3.08e-13
df['T_LATE_DELIVERIES']                 -121.9967     80.685     -1.512      0.131    -280.237      36.244
df['T_AVG_PREP_VID_TIME']               -434.7108    150.174     -2.895      0.004    -

In [12]:
######################################################################################################################
# Create the standardized file to use in linear models ###############################################################
######################################################################################################################


#Creating df with x_variables
x_variables = ['CROSS_SELL_SUCCESS',
               'TOTAL_MEALS_ORDERED',
               'UNIQUE_MEALS_PURCH',
               'CONTACTS_W_CUSTOMER_SERVICE',
               'PRODUCT_CATEGORIES_VIEWED',
               'CANCELLATIONS_AFTER_NOON',
               'MOBILE_LOGINS',
               'LATE_DELIVERIES',
               'FOLLOWED_RECOMMENDATIONS_PCT',
               'AVG_PREP_VID_TIME',
               'LARGEST_ORDER_SIZE',
               'MASTER_CLASSES_ATTENDED',
               'MEDIAN_MEAL_RATING',
               'AVG_CLICKS_PER_VISIT',
               'AVG_PREP_VID_TIME_PER_ORDER',
               'AVG_CLICKS_PER_VISIT_PER_ORDER',
               'NUMBER_OF_RECOMMENDATION',
               'T_UNIQUE_MEALS_PURCH',
               'T_PRODUCT_CATEGORIES_VIEWED',
               'T_AVG_TIME_PER_SITE_VISIT',
               'T_PC_LOGINS',
               'T_MOBILE_LOGINS',
               'T_EARLY_DELIVERIES',
               'T_LATE_DELIVERIES',
               'T_AVG_PREP_VID_TIME',
               'T_AVG_CLICKS_PER_VISIT',
               'T_TOTAL_PHOTOS_VIEWED',
               'T_AVG_PREP_VID_TIME_PER_ORDER',
               'T_AVG_CLICKS_PER_VISIT_PER_ORDER',
               'UNIQUE_MEALS_PURCH_C',
               'AVG_CLICKS_PER_VISIT_C',
               'AVG_PREP_VID_TIME_C',
               'TOTAL_PHOTOS_VIEWED_C']     
                
#dropping

#df_explanatory= df_explanatory.drop(['REVENUE'],axis=1)

# preparing explanatory variable data
appen_data   = df_explanatory.loc[ : , x_variables]

# preparing response variable data
appen_target = df_explanatory.loc[:, 'REVENUE']

# preparing training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
            appen_data,
            appen_target,
            test_size = 0.25,
            random_state = 222)



In [13]:
######################################################################################################################
# Create the standardized file to use in linear models ###############################################################
######################################################################################################################

# INSTANTIATING a StandardScaler() object
scaler = StandardScaler()

# FITTING the scaler with housing_data
scaler.fit(appen_data)

# TRANSFORMING our data after fit
X_scaled = scaler.transform(appen_data)

# converting scaled data into a DataFrame
X_scaled_df = pd.DataFrame(X_scaled)

# checking the results
X_scaled_df.describe().round(2)

X_scaled_df.columns = appen_data.columns

X_scaled_df

# this is the exact code we were using before
X_train, X_test, y_train, y_test = train_test_split(
            X_scaled_df,
            appen_target,
            test_size = 0.25,
            random_state = 222)

In [14]:
# adding labels to the scaled DataFrame
X_scaled_df.columns = appen_data.columns

#  Checking pre- and post-scaling of the data
print(f"""
Dataset BEFORE Scaling
----------------------
{pd.np.var(appen_data)}


Dataset AFTER Scaling
----------------------
{pd.np.var(X_scaled_df)}
""")


Dataset BEFORE Scaling
----------------------
CROSS_SELL_SUCCESS                     0.218020
TOTAL_MEALS_ORDERED                 3057.599946
UNIQUE_MEALS_PURCH                     6.257663
CONTACTS_W_CUSTOMER_SERVICE            5.201168
PRODUCT_CATEGORIES_VIEWED              9.261178
CANCELLATIONS_AFTER_NOON               0.186736
MOBILE_LOGINS                          0.278218
LATE_DELIVERIES                        7.528947
FOLLOWED_RECOMMENDATIONS_PCT         706.023186
AVG_PREP_VID_TIME                   2443.767133
LARGEST_ORDER_SIZE                     2.404278
MASTER_CLASSES_ATTENDED                0.411780
MEDIAN_MEAL_RATING                     0.572040
AVG_CLICKS_PER_VISIT                   5.444177
AVG_PREP_VID_TIME_PER_ORDER            4.769608
AVG_CLICKS_PER_VISIT_PER_ORDER         0.098951
NUMBER_OF_RECOMMENDATION             881.777907
T_UNIQUE_MEALS_PURCH                   0.009668
T_PRODUCT_CATEGORIES_VIEWED            0.076318
T_AVG_TIME_PER_SITE_VISIT              0.

In [15]:
######################################################################################################################
## LinearRegression ##################################################################################################
######################################################################################################################

# INSTANTIATING 
lr = sklearn.linear_model.LinearRegression()

# FITTING 
lr_fit = lr.fit(X_train, y_train)

# PREDICTING 
lr_pred = lr_fit.predict(X_test)

# SCORING
print('Training Score:', lr.score(X_train, y_train).round(4))
print('Testing Score:',  lr.score(X_test, y_test).round(4))

#saving
lr_train_score = lr.score(X_train, y_train).round(4)
lr_test_score  = lr.score(X_test, y_test).round(4)

Training Score: 0.7003
Testing Score: 0.635


In [16]:
######################################################################################################################
## Ridge #############################################################################################################
######################################################################################################################

# INSTANTIATING 
ridge_model = sklearn.linear_model.Ridge()

# FITTING 
ridge_fit = ridge_model.fit(X_train, y_train)


# PREDICTING
ridge_pred = ridge_model.predict(X_test)

print('Training Score:', ridge_model.score(X_train, y_train).round(4))
print('Testing Score:',  ridge_model.score(X_test, y_test).round(4))


# saving
ridge_train_score = ridge_model.score(X_train, y_train).round(4)
ridge_test_score  = ridge_model.score(X_test, y_test).round(4)

Training Score: 0.7003
Testing Score: 0.6348


In [17]:
######################################################################################################################
## Lasso #############################################################################################################
######################################################################################################################

# INSTANTIATING 
lasso_model = sklearn.linear_model.Lasso()

# FITTING 
lasso_fit = lasso_model.fit(X_train, y_train)


# PREDICTING 
lasso_pred = lasso_model.predict(X_test)

print('Training Score:', lasso_model.score(X_train, y_train).round(4))
print('Testing Score:',  lasso_model.score(X_test, y_test).round(4))


# saving
lasso_train_score = lasso_model.score(X_train, y_train).round(4)
lasso_test_score  = lasso_model.score(X_test, y_test).round(4)

Training Score: 0.7001
Testing Score: 0.6336


In [18]:
######################################################################################################################
## SGD ##############################################################################################################
######################################################################################################################

# INSTANTIATING 
SGD_model = sklearn.linear_model.SGDRegressor()

# FITTING 
SGD_fit = SGD_model.fit(X_train, y_train)


# PREDICTING 
SGD_pred = lasso_model.predict(X_test)

print('Training Score:', SGD_model.score(X_train, y_train).round(4))
print('Testing Score:',  SGD_model.score(X_test, y_test).round(4))

# saving
SGD_train_score = SGD_model.score(X_train, y_train).round(4)
SGD_test_score  = SGD_model.score(X_test, y_test).round(4)

Training Score: 0.6983
Testing Score: 0.63


In [19]:
######################################################################################################################
## Elastic Net CV model object #######################################################################################
######################################################################################################################

# INSTANTIATING 
elastic_model = sklearn.linear_model.ElasticNetCV()

# FITTING 
elastic_fit = elastic_model.fit(X_train, y_train)


# PREDICTING 
elastic_pred = elastic_model.predict(X_test)

print('Training Score:', elastic_model.score(X_train, y_train).round(4))
print('Testing Score:',  elastic_model.score(X_test, y_test).round(4))

# saving
elastic_train_score = elastic_model.score(X_train, y_train).round(4)
elastic_test_score  = elastic_model.score(X_test, y_test).round(4)

Training Score: 0.6569
Testing Score: 0.5811


In [20]:
######################################################################################################################
## Elastic Net CV model object #######################################################################################
######################################################################################################################

# INSTANTIATING
elastic_model = sklearn.ensemble.GradientBoostingRegressor(n_estimators=150, max_depth=2, min_samples_leaf=120)

# FITTING 
elastic_fit = elastic_model.fit(X_train, y_train)


# PREDICTING 
elastic_pred = elastic_model.predict(X_test)

print('Training Score:', elastic_model.score(X_train, y_train).round(4))
print('Testing Score:',  elastic_model.score(X_test, y_test).round(4))

# saving
elastic_train_score = elastic_model.score(X_train, y_train).round(4)
elastic_test_score  = elastic_model.score(X_test, y_test).round(4)

Training Score: 0.8215
Testing Score: 0.7347


Comparing linear regression, lasso, SGD Regression and Elastic Net CV model object, Lasso provided the best testing score.